In [ ]:
USE ROLE ACCOUNTADMIN;
-- Next create a new database and schema,
CREATE OR REPLACE DATABASE ML_LINEAGE_DATABASE;
CREATE OR REPLACE SCHEMA ML_LINEAGE_SCHEMA;


In [ ]:
use schema ML_LINEAGE_DATABASE.ML_LINEAGE_SCHEMA;

In [ ]:
create or replace stage LINEAGE_STAGE DIRECTORY=(enable=TRUE);

In [ ]:
# Make sure snowflake-ml-python is included in "Packages" up top
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", "name":"aiml_notebooks_lineage", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"notebook"}}

CURRENT_DB = session.get_current_database()
CURRENT_SCHEMA = session.get_current_schema()

In [ ]:
# creates 
# external stage: FEATURE_STORE_TEMP_STAGE
# table: winedata (populated w/1.6K rows) 
# file format: FEATURE_STORE_TEMP_FORMAT
from snowflake.ml.feature_store.examples.example_helper import ExampleHelper

example_helper = ExampleHelper(session, CURRENT_DB, CURRENT_SCHEMA)
source_tables = example_helper.load_example('wine_quality_features')

In [ ]:
-- should be 1,599
select count(distinct wine_id) from winedata;

In [ ]:
create or replace temporary table wine_attributes_tmp as
select 
seq4() as wine_id,
case 
when wine_id between 0 and 999 then 'Red' 
when wine_id between 1000 and 1299 then 'White'
when wine_id > 1299 then 'Rose'
end as wine_color, 
uniform(1, 10, RANDOM(12)) as CountryId,
uniform(1, 50, RANDOM(12)) as RegionId,
uniform(2005, 2024, RANDOM(12)) as VintageYear

FROM TABLE(GENERATOR(ROWCOUNT => 1599));

create or replace table wine_attributes 
(c1 variant);
    -- like wine_attributes_tmp;

In [ ]:
COPY INTO @LINEAGE_STAGE
FROM wine_attributes_tmp
  FILE_FORMAT=(TYPE=parquet);